In [ ]:
import numpy as np
import torch
from torch import Tensor
import torch.nn.functional as f
from torch import nn
import scipy.io as sio
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import TensorDataset, DataLoader
from multiprocessing import cpu_count
import torch.nn.functional as F
import torch.optim as optim


In [ ]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [ ]:
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, chi2, f_classif


In [ ]:
'''%load_ext tensorboard'
%tensorboard — logdir logs'''

"%load_ext tensorboard'\n%tensorboard — logdir logs"

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# New Section

In [ ]:
torch.cuda.empty_cache()
#load the ECG file
src_dataset = sio.loadmat('/content/drive/MyDrive/ECG2(withDA).mat')
#load the labels
label=pd.read_csv('/content/drive/MyDrive/label.csv',header=None)

In [ ]:
#print(src_dataset)
testdata = src_dataset['ECG'] # use the key for data here
X=testdata['Data']

In [ ]:
X=np.array(X[0])



In [ ]:
X = np.vstack(X[:,]).astype(np.float)

In [ ]:
#X = X[:,3350:4000]

In [ ]:
X=torch.from_numpy(X)

In [ ]:
X.dtype

torch.float64

In [ ]:
label=np.array(label, dtype=object)
label=np.array(label)
label= np.vstack(label[:,]).astype(np.float)


In [ ]:
df = pd.DataFrame(data =X)

In [ ]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,3960,3961,3962,3963,3964,3965,3966,3967,3968,3969,3970,3971,3972,3973,3974,3975,3976,3977,3978,3979,3980,3981,3982,3983,3984,3985,3986,3987,3988,3989,3990,3991,3992,3993,3994,3995,3996,3997,3998,3999
0,"tensor(0.0160, dtype=torch.float64)","tensor(0.0164, dtype=torch.float64)","tensor(0.0169, dtype=torch.float64)","tensor(0.0174, dtype=torch.float64)","tensor(0.0180, dtype=torch.float64)","tensor(0.0187, dtype=torch.float64)","tensor(0.0188, dtype=torch.float64)","tensor(0.0189, dtype=torch.float64)","tensor(0.0191, dtype=torch.float64)","tensor(0.0194, dtype=torch.float64)","tensor(0.0196, dtype=torch.float64)","tensor(0.0194, dtype=torch.float64)","tensor(0.0191, dtype=torch.float64)","tensor(0.0189, dtype=torch.float64)","tensor(0.0188, dtype=torch.float64)","tensor(0.0187, dtype=torch.float64)","tensor(0.0180, dtype=torch.float64)","tensor(0.0174, dtype=torch.float64)","tensor(0.0168, dtype=torch.float64)","tensor(0.0163, dtype=torch.float64)","tensor(0.0158, dtype=torch.float64)","tensor(0.0148, dtype=torch.float64)","tensor(0.0138, dtype=torch.float64)","tensor(0.0128, dtype=torch.float64)","tensor(0.0120, dtype=torch.float64)","tensor(0.0112, dtype=torch.float64)","tensor(0.0099, dtype=torch.float64)","tensor(0.0086, dtype=torch.float64)","tensor(0.0074, dtype=torch.float64)","tensor(0.0063, dtype=torch.float64)","tensor(0.0053, dtype=torch.float64)","tensor(0.0038, dtype=torch.float64)","tensor(0.0024, dtype=torch.float64)","tensor(0.0010, dtype=torch.float64)","tensor(-0.0002, dtype=torch.float64)","tensor(-0.0013, dtype=torch.float64)","tensor(-0.0028, dtype=torch.float64)","tensor(-0.0043, dtype=torch.float64)","tensor(-0.0056, dtype=torch.float64)","tensor(-0.0068, dtype=torch.float64)",...,"tensor(0.1613, dtype=torch.float64)","tensor(0.1398, dtype=torch.float64)","tensor(0.1258, dtype=torch.float64)","tensor(0.1267, dtype=torch.float64)","tensor(0.1458, dtype=torch.float64)","tensor(0.1807, dtype=torch.float64)","tensor(0.2611, dtype=torch.float64)","tensor(0.3193, dtype=torch.float64)","tensor(0.3253, dtype=torch.float64)","tensor(0.2548, dtype=torch.float64)","tensor(0.0948, dtype=torch.float64)","tensor(-0.2069, dtype=torch.float64)","tensor(-0.5564, dtype=torch.float64)","tensor(-0.9033, dtype=torch.float64)","tensor(-1.1942, dtype=torch.float64)","tensor(-1.3821, dtype=torch.float64)","tensor(-1.3838, dtype=torch.float64)","tensor(-1.2546, dtype=torch.float64)","tensor(-1.0174, dtype=torch.float64)","tensor(-0.7091, dtype=torch.float64)","tensor(-0.3737, dtype=torch.float64)","tensor(-0.0846, dtype=torch.float64)","tensor(0.1591, dtype=torch.float64)","tensor(0.3383, dtype=torch.float64)","tensor(0.4436, dtype=torch.float64)","tensor(0.4760, dtype=torch.float64)","tensor(0.4463, dtype=torch.float64)","tensor(0.3689, dtype=torch.float64)","tensor(0.2640, dtype=torch.float64)","tensor(0.1523, dtype=torch.float64)","tensor(0.0528, dtype=torch.float64)","tensor(-0.0075, dtype=torch.float64)","tensor(-0.0385, dtype=torch.float64)","tensor(-0.0434, dtype=torch.float64)","tensor(-0.0297, dtype=torch.float64)","tensor(-0.0071, dtype=torch.float64)","tensor(-3.7935e-05, dtype=torch.float64)","tensor(0.0029, dtype=torch.float64)","tensor(0.0011, dtype=torch.float64)","tensor(-0.0046, dtype=torch.float64)"
1,"tensor(-0.1149, dtype=torch.float64)","tensor(-0.1224, dtype=torch.float64)","tensor(-0.1289, dtype=torch.float64)","tensor(-0.1332, dtype=torch.float64)","tensor(-0.1342, dtype=torch.float64)","tensor(-0.1316, dtype=torch.float64)","tensor(-0.1249, dtype=torch.float64)","tensor(-0.1155, dtype=torch.float64)","tensor(-0.1048, dtype=torch.float64)","tensor(-0.0943, dtype=torch.float64)","tensor(-0.0856, dtype=torch.float64)","tensor(-0.0811, dtype=torch.float64)","tensor(-0.0802, dtype=torch.float64)","tensor(-0.0828, dtype=torch.float64)","tensor(-0.0882, dtype=torch.float64)","tensor(-0.0954, dtype=t

In [ ]:
label = pd.DataFrame(data = label)


In [ ]:
label

,0
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
1268,1.0
1269,1.0
1270,1.0
1271,1.0


In [ ]:
df1 = pd.merge(df,label, left_index = True, right_index=True)


In [ ]:
df1

,0_x,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,3961,3962,3963,3964,3965,3966,3967,3968,3969,3970,3971,3972,3973,3974,3975,3976,3977,3978,3979,3980,3981,3982,3983,3984,3985,3986,3987,3988,3989,3990,3991,3992,3993,3994,3995,3996,3997,3998,3999,0_y
0,"tensor(0.0160, dtype=torch.float64)","tensor(0.0164, dtype=torch.float64)","tensor(0.0169, dtype=torch.float64)","tensor(0.0174, dtype=torch.float64)","tensor(0.0180, dtype=torch.float64)","tensor(0.0187, dtype=torch.float64)","tensor(0.0188, dtype=torch.float64)","tensor(0.0189, dtype=torch.float64)","tensor(0.0191, dtype=torch.float64)","tensor(0.0194, dtype=torch.float64)","tensor(0.0196, dtype=torch.float64)","tensor(0.0194, dtype=torch.float64)","tensor(0.0191, dtype=torch.float64)","tensor(0.0189, dtype=torch.float64)","tensor(0.0188, dtype=torch.float64)","tensor(0.0187, dtype=torch.float64)","tensor(0.0180, dtype=torch.float64)","tensor(0.0174, dtype=torch.float64)","tensor(0.0168, dtype=torch.float64)","tensor(0.0163, dtype=torch.float64)","tensor(0.0158, dtype=torch.float64)","tensor(0.0148, dtype=torch.float64)","tensor(0.0138, dtype=torch.float64)","tensor(0.0128, dtype=torch.float64)","tensor(0.0120, dtype=torch.float64)","tensor(0.0112, dtype=torch.float64)","tensor(0.0099, dtype=torch.float64)","tensor(0.0086, dtype=torch.float64)","tensor(0.0074, dtype=torch.float64)","tensor(0.0063, dtype=torch.float64)","tensor(0.0053, dtype=torch.float64)","tensor(0.0038, dtype=torch.float64)","tensor(0.0024, dtype=torch.float64)","tensor(0.0010, dtype=torch.float64)","tensor(-0.0002, dtype=torch.float64)","tensor(-0.0013, dtype=torch.float64)","tensor(-0.0028, dtype=torch.float64)","tensor(-0.0043, dtype=torch.float64)","tensor(-0.0056, dtype=torch.float64)","tensor(-0.0068, dtype=torch.float64)",...,"tensor(0.1398, dtype=torch.float64)","tensor(0.1258, dtype=torch.float64)","tensor(0.1267, dtype=torch.float64)","tensor(0.1458, dtype=torch.float64)","tensor(0.1807, dtype=torch.float64)","tensor(0.2611, dtype=torch.float64)","tensor(0.3193, dtype=torch.float64)","tensor(0.3253, dtype=torch.float64)","tensor(0.2548, dtype=torch.float64)","tensor(0.0948, dtype=torch.float64)","tensor(-0.2069, dtype=torch.float64)","tensor(-0.5564, dtype=torch.float64)","tensor(-0.9033, dtype=torch.float64)","tensor(-1.1942, dtype=torch.float64)","tensor(-1.3821, dtype=torch.float64)","tensor(-1.3838, dtype=torch.float64)","tensor(-1.2546, dtype=torch.float64)","tensor(-1.0174, dtype=torch.float64)","tensor(-0.7091, dtype=torch.float64)","tensor(-0.3737, dtype=torch.float64)","tensor(-0.0846, dtype=torch.float64)","tensor(0.1591, dtype=torch.float64)","tensor(0.3383, dtype=torch.float64)","tensor(0.4436, dtype=torch.float64)","tensor(0.4760, dtype=torch.float64)","tensor(0.4463, dtype=torch.float64)","tensor(0.3689, dtype=torch.float64)","tensor(0.2640, dtype=torch.float64)","tensor(0.1523, dtype=torch.float64)","tensor(0.0528, dtype=torch.float64)","tensor(-0.0075, dtype=torch.float64)","tensor(-0.0385, dtype=torch.float64)","tensor(-0.0434, dtype=torch.float64)","tensor(-0.0297, dtype=torch.float64)","tensor(-0.0071, dtype=torch.float64)","tensor(-3.7935e-05, dtype=torch.float64)","tensor(0.0029, dtype=torch.float64)","tensor(0.0011, dtype=torch.float64)","tensor(-0.0046, dtype=torch.float64)",0.0
1,"tensor(-0.1149, dtype=torch.float64)","tensor(-0.1224, dtype=torch.float64)","tensor(-0.1289, dtype=torch.float64)","tensor(-0.1332, dtype=torch.float64)","tensor(-0.1342, dtype=torch.float64)","tensor(-0.1316, dtype=torch.float64)","tensor(-0.1249, dtype=torch.float64)","tensor(-0.1155, dtype=torch.float64)","tensor(-0.1048, dtype=torch.float64)","tensor(-0.0943, dtype=torch.float64)","tensor(-0.0856, dtype=torch.float64)","tensor(-0.0811, dtype=torch.float64)","tensor(-0.0802, dtype=torch.float64)","tensor(-0.0828, dtype=torch.float64)","tensor(-0.0882, dtype=torch.float64)","tensor(-0.0954, dtype=torch.float64)","tensor(-0.1043, d

In [ ]:
df1.groupby('0_y').nunique()

SystemError: ignored

In [ ]:
a = df1.loc[df1['0_y']== 0,:]

In [ ]:
b = df1.loc[df1['0_y']== 1,:]
c = df1.loc[df1['0_y']== 2,:]

In [ ]:
print(a.shape)
print(b.shape)
print(c.shape)

In [ ]:
#take 250 rows from each of the section
a = a[1:250]
b = b[1:250]
c = c[1:250]

In [ ]:
#del a['0_y']
#del b['0_y']
#del c['0_y']

In [ ]:
a = a.reset_index(drop = True)

In [ ]:
b =b.reset_index(drop = True)
c = c.reset_index(drop = True)

In [ ]:
df2 = pd.concat([a,b])


In [ ]:
df2 = pd.concat([df2,c])

In [ ]:
df2 = df2.reset_index(drop = True)

In [ ]:
label_1 = df2['0_y']

In [ ]:
del df2['0_y']

In [ ]:
label_1 = label_1.to_numpy()

In [ ]:
Y = torch.tensor(label_1)

In [ ]:
Y.shape

In [ ]:
Y = torch.tensor(label)

In [ ]:
df2

In [ ]:
X = df2.to_numpy()

In [ ]:
print(X.shape)
print(Y.shape)

In [ ]:
#from sklearn.datasets import make_classification

In [ ]:
X,y = make_classification(
    n_features = 35,n_classes = 3, random_state = 42, n_informative=3, n_redundant =0, n_clusters_per_class=2
)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
anova_filter = SelectKBest(f_classif, k=3)

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.svm import LinearSVC

In [ ]:
clf = LinearSVC()

In [ ]:
anova_svm = make_pipeline(anova_filter, clf)
anova_svm.fit(X_train, y_train)

In [ ]:
pca = PCA(n_components=4000, svd_solver= 'full')

In [ ]:
pca.fit(X)

In [ ]:
pca.explained_variance_ratio_

In [ ]:
pca.singular_values_

In [ ]:
X_pca = pca.singular_values_

In [ ]:
X_pca = X_pca.reshape(1273,1)

In [ ]:
X_new = SelectKBest(chi2, k='all').fit_transform(X_pca, label)

In [ ]:
X_new.shape

In [ ]:
X.shape

torch.Size([1273, 4000])

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)

In [ ]:
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train, Y_train, test_size = 0.25, stratify = Y_train)

In [ ]:
X_train.shape, X_valid.shape, Y_train.shape, Y_valid.shape

(torch.Size([763, 4000]),
 torch.Size([255, 4000]),
 torch.Size([763]),
 torch.Size([255]))

In [ ]:
X_train.dtype

torch.float64

In [ ]:
X_train.shape, X_valid.shape, Y_train.shape, Y_valid.shape

(torch.Size([763, 4000]),
 torch.Size([255, 4000]),
 torch.Size([763]),
 torch.Size([255]))

In [ ]:
Y_valid.shape

torch.Size([255])

In [ ]:
Y_valid = torch.squeeze(Y_valid)
Y_valid.shape

torch.Size([255])

In [ ]:
Y_train = torch.squeeze(Y_train)
Y_test = torch.squeeze(Y_test)

In [ ]:
#X_train = torch.tensor(X_train)
#X_valid = torch.tensor(X_valid)
#X_test = torch.tensor(X_test)

X_train, X_test, X_valid = [torch.tensor(arr) for arr in (X_train, X_test, X_valid )]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """


In [ ]:
Y_train[1]

tensor(0., dtype=torch.float64)

In [ ]:
Y_train, Y_test, Y_valid = [torch.tensor(arr) for arr in (Y_train, Y_test, Y_valid)]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


In [ ]:
train_ds = TensorDataset(X_train, Y_train)

In [ ]:
test_ds = TensorDataset(X_test, Y_test)
valid_ds = TensorDataset(X_valid, Y_valid)


In [ ]:
train_dl = DataLoader(train_ds, batch_size = 32, shuffle = True)
test_dl = DataLoader(test_ds, batch_size = 32, shuffle = True)
valid_dl = DataLoader(valid_ds, batch_size=32, shuffle=True)

In [ ]:
print(len(train_dl), len(test_dl), len(valid_dl))

24 8 8


In [ ]:
def create_datasets(X, Y):
    enc = LabelEncoder()
    Y_enc = enc.fit_transform(Y)
    print("len of X and Y is",X.shape,Y.shape)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify = Y) 
    X_train, X_valid, Y_train, Y_valid = train_test_split(X_train, Y_train, test_size=0.25, stratify = Y_train) 
    #squeeze labelled data to make it 1 dimension array
    Y_train=torch.squeeze(Y_train)
    Y_valid=torch.squeeze(Y_valid)
    Y_test=torch.squeeze(Y_test)
    print("X_train, X_valid, X_test shape before list comprehension",X_train.shape, X_valid.shape, X_test.shape)
    print("X_train, X_valid, X_test dtype before list comprehension",X_train.dtype, X_valid.dtype, X_test.dtype)
    print("Y_train, Y_valid, Y_test dtype before list comprehension",Y_train.dtype, Y_valid.dtype, Y_test.dtype)
    #the tensors created have dtype float64 but the models have dtypes float32 and its easier to change dtypes of the tensors instead of a model
    X_train, X_valid, X_test = [torch.tensor(arr, dtype=torch.float32) for arr in (X_train, X_valid,X_test)]
    Y_train, Y_valid,Y_test = [torch.tensor(arr, dtype=torch.long) for arr in (Y_train, Y_valid,Y_test)]
    print("X_train, X_valid, X_test shape after list comprehension",X_train.shape, X_valid.shape, X_test.shape)
    print("X_train, X_valid, X_test dtype before list comprehension",X_train.dtype, X_valid.dtype, X_test.dtype)
    print("Y_train, Y_valid, Y_test dtype before list comprehension",Y_train.dtype, Y_valid.dtype, Y_test.dtype)
    #convert training and testing data into Tensor Dataset
    train_ds = TensorDataset(X_train, Y_train)
    valid_ds = TensorDataset(X_valid, Y_valid)
    test_ds = TensorDataset(X_test,Y_test)
    print("train_ds, valid_ds, test_ds length of datasets respectively",len(train_ds), len(valid_ds), len(test_ds))
    return train_ds, valid_ds,test_ds, enc

In [ ]:
def create_loaders(train_ds, valid_ds,test_ds, bs, jobs=0):
    train_dl = DataLoader(train_ds, bs, shuffle=True, num_workers=jobs)
    valid_dl = DataLoader(valid_ds, bs, shuffle=True, num_workers=jobs)
    test_dl = DataLoader(test_ds, bs, shuffle=True, num_workers=jobs)
    print("lenth of data loaders train_dl, valid_dl, test_dl",len(train_dl), len(valid_dl), len(test_dl))
    return train_dl, valid_dl, test_dl

In [ ]:
def accuracy(output, target):
    return (output.argmax(dim=1) == target).float().mean().item()

In [ ]:
class LSTMClassifier(nn.Module):
    """Very simple implementation of LSTM-based time-series classifier."""

    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.batch_size = None
        self.hidden = None
        
    def forward(self, x):
        h0, c0 = self.init_hidden(x)
        print(x.size())
        out, (hn, cn) = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out


    def init_hidden(self, x):
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
        print(h0.shape)
        print(x.size(0))
        print(layer_dim)
        return [t.to(device) for t in (h0, c0)]


In [ ]:
train_ds, val_ds, test_data, enc = create_datasets(X, Y)

len of X and Y is torch.Size([1273, 4000]) torch.Size([1273, 1])
X_train, X_valid, X_test shape before list comprehension torch.Size([763, 4000]) torch.Size([255, 4000]) torch.Size([255, 4000])
X_train, X_valid, X_test dtype before list comprehension torch.float64 torch.float64 torch.float64
Y_train, Y_valid, Y_test dtype before list comprehension torch.float64 torch.float64 torch.float64
X_train, X_valid, X_test shape after list comprehension torch.Size([763, 4000]) torch.Size([255, 4000]) torch.Size([255, 4000])
X_train, X_valid, X_test dtype before list comprehension torch.float32 torch.float32 torch.float32
Y_train, Y_valid, Y_test dtype before list comprehension torch.int64 torch.int64 torch.int64
train_ds, valid_ds, test_ds length of datasets respectively 763 255 255


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  app.launch_new_instance()


In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
bs =32
print(f'Creating data loaders with batch size: {bs}')
train_dl, val_dl,test_data = create_loaders(train_ds, val_ds,test_data, bs, jobs=cpu_count())

Creating data loaders with batch size: 32
lenth of data loaders train_dl, valid_dl, test_dl 24 8 8


In [ ]:

input_dim = 1
hidden_dim = 100
layer_dim = 1
output_dim = 1
seq_dim = 4000
lr = 0.01
n_epochs = 101
iterations_per_epoch = len(train_dl)
best_acc = 0
patience, trials = 100, 0




In [ ]:
model = LSTMClassifier(input_dim,hidden_dim,layer_dim,output_dim)
model = model.to(device)
criteria = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr)

In [ ]:
model

LSTMClassifier(
  (lstm): LSTM(1, 100, batch_first=True)
  (fc): Linear(in_features=100, out_features=1, bias=True)
)

In [ ]:
model.train()

LSTMClassifier(
  (lstm): LSTM(1, 100, batch_first=True)
  (fc): Linear(in_features=100, out_features=1, bias=True)
)

In [ ]:
#opt = torch.optim.Adam(model.parameters(), lr=0.001)
#criterion = nn.CrossEntropyLoss()

In [ ]:
'''class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=100, output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size,batch_first=True)

        self.linear = nn.Linear(hidden_layer_size, output_size)
        
        self.fc = nn.Linear(hidden_layer_size, output_size)

        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size),
                            torch.zeros(1,1,self.hidden_layer_size))

    def forward(self, input_seq):
        h0 = torch.zeros(1, input_seq.size(0), self.hidden_layer_size).to(device) 
        c0 = torch.zeros(1, input_seq.size(0), self.hidden_layer_size).to(device)
        lstm_out, _ = self.lstm(input_seq, (h0,c0))
        lstm_out = self.fc(lstm_out[:, -1, :])
        #predictions = self.linear(lstm_out.view(len(input_seq), -1))
        #print("predictions",predictions)
        return lstm_out'''
    '''def init_hidden(self, x):
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
        
        print('hidden layer')
        print(h0.shape)
        print(x.size(0))
        print(layer_dim)
        return [t.to(device) for t in (h0, c0)]'''

In [ ]:
'''trn_ds, val_ds, tst_data, enc = create_datasets(X, Y)
bs =50
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')'''


In [ ]:
'''print(f'Creating data loaders with batch size: {bs}')
trn_dl, val_dl,tst_data = create_loaders(trn_ds, val_ds,tst_data, bs, jobs=cpu_count())
model = LSTM(1,500,1)
layer_dim = 1'''

In [ ]:
'''model = model.to(device)'''

In [ ]:
'''seq_dim = 4000
lr = 0.001
n_epochs = 101
iterations_per_epoch = len(trn_dl)
best_acc = 0
patience, trials = 100, 0
loss_function = nn.CrossEntropyLoss()
layer_dim = 1
output_dim = 1'''

In [ ]:
import tensorflow as tf
from tensorflow import summary
%load_ext tensorboard
train_log_dir ='logs/train'
test_log_dir = 'logs/test'
train_summary_writer = summary.create_file_writer(train_log_dir)
test_summary_writer = summary.create_file_writer(test_log_dir)

In [ ]:
@tf.function 
def my_func(step,loss):
  with train_summary_writer.as_default():
    tf.summary.scalar("loss", loss.item(), step)

In [ ]:
writer = SummaryWriter('runs')

In [ ]:
tensorboard --logdir=runs

In [ ]:
print('Start model training')
globaliter = 0

Start model training


In [ ]:
train_ds[0]

(tensor([0.0316, 0.0339, 0.0343,  ..., 0.2084, 0.1975, 0.1701]), tensor(0))

In [ ]:
for epoch in range(1):
  for i, (x,y) in enumerate(train_ds):
    print("i is", i)
    print("x_batch, y_batch",x,y)
    print("x_batch", x.shape)
    print("x_ dimesns", x.ndim)
    print("y_batch", y.shape)
    print("y dimension", y.ndim)
  print("epoch is", epoch)
  print("train_dl length is",len(train_ds))

i is 0
x_batch, y_batch tensor([0.0316, 0.0339, 0.0343,  ..., 0.2084, 0.1975, 0.1701]) tensor(0)
x_batch torch.Size([4000])
x_ dimesns 1
y_batch torch.Size([])
y dimension 0
i is 1
x_batch, y_batch tensor([ 0.0088,  0.0104,  0.0111,  ..., -0.0026, -0.0090, -0.0149]) tensor(2)
x_batch torch.Size([4000])
x_ dimesns 1
y_batch torch.Size([])
y dimension 0
i is 2
x_batch, y_batch tensor([ 0.1967,  0.1726,  0.1211,  ..., -0.0065, -0.0061, -0.0057]) tensor(0)
x_batch torch.Size([4000])
x_ dimesns 1
y_batch torch.Size([])
y dimension 0
i is 3
x_batch, y_batch tensor([ 0.0007, -0.0030, -0.0059,  ..., -0.0633, -0.0623, -0.0597]) tensor(2)
x_batch torch.Size([4000])
x_ dimesns 1
y_batch torch.Size([])
y dimension 0
i is 4
x_batch, y_batch tensor([ 0.0073,  0.0099,  0.0051,  ..., -0.0583, -0.0568, -0.0518]) tensor(1)
x_batch torch.Size([4000])
x_ dimesns 1
y_batch torch.Size([])
y dimension 0
i is 5
x_batch, y_batch tensor([-0.0342, -0.0454, -0.0568,  ...,  0.0880,  0.0801,  0.0671]) tensor(0)
x_b

In [ ]:
  for i,(x_batch,y_batch) in enumerate(train_dl):
    if i in range(5):
      print("i is", i)
      print("x_batch, y_batch",x_batch,y_batch)
      print("x_batch shape", x_batch.shape)
      print("x_batch size", x_batch.size(0))
      print("x_batch", x_batch.ndim)
      print("y_batch", y_batch.shape)
      print("y_batch", y_batch.ndim)
      x_batch=torch.unsqueeze(x_batch,2)
      print("x_batch, y_batch",x_batch,y_batch)
      print("x_batch", x_batch.shape)
      print("x_batch", x_batch.ndim)
      print("y_batch", y_batch.shape)
      print("y_batch", y_batch.ndim)

    else:
      break


i is 0
x_batch, y_batch tensor([[ 8.8749e-03,  9.8668e-03,  1.1456e-02,  ...,  2.9131e-03,
          1.4776e-03,  2.6194e-04],
        [-6.9665e-02, -6.4873e-02, -5.9936e-02,  ..., -1.1124e-01,
         -1.0493e-01, -8.5985e-02],
        [-4.9185e-02, -4.8871e-02, -5.2081e-02,  ...,  3.2429e-02,
          3.9078e-02,  4.9032e-02],
        ...,
        [-8.0190e-02, -9.1382e-02, -1.0012e-01,  ..., -1.8947e-02,
         -1.2319e-02, -5.5691e-03],
        [-3.5461e-02, -3.2061e-02, -2.8606e-02,  ..., -2.8495e-03,
         -2.6729e-04,  2.4288e-03],
        [-3.6676e-01, -3.9568e-01, -4.1818e-01,  ..., -5.7608e-02,
         -6.0934e-02, -6.3642e-02]]) tensor([0, 0, 1, 0, 1, 1, 2, 1, 0, 0, 2, 2, 2, 1, 0, 0, 0, 0, 2, 2, 0, 0, 0, 2,
        0, 1, 0, 1, 0, 1, 2, 0])
x_batch shape torch.Size([32, 4000])
x_batch size 32
x_batch 2
y_batch torch.Size([32])
y_batch 1
x_batch, y_batch tensor([[[ 8.8749e-03],
         [ 9.8668e-03],
         [ 1.1456e-02],
         ...,
         [ 2.9131e-03],
      

In [ ]:
for i, (x_batch,y_batch) in enumerate(train_dl):
  if i in range(3):
    output = model(x_batch)
  else:
    break

torch.Size([1, 32, 100])
32
1
torch.Size([32, 4000])


RuntimeError: ignored

In [ ]:
for epoch in range(1, n_epochs + 1):

    for i, (x_batch, y_batch) in enumerate(train_dl):
        globaliter +=1
        #model.train() optional, it is done to set the model in training mode which is by default
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        print('shape of the x_input batch', x_batch.shape)
        print('shape of the y_input batch', y_batch.shape)
        opt.zero_grad()
        #added the dimesnion here at 2nd position which makes the shape of tensor (32,4000,1)
        x_batch=torch.unsqueeze(x_batch,2)
        out = model(x_batch)
        y_batch=torch.unsqueeze(y_batch,0)
        print(y_batch.dtype)
        y_batch = y_batch.to(torch.float32)

        out = out.to(torch.float32)
        out=torch.transpose(out,1,0)
        loss = loss_function(out, torch.max(y_batch, 1)[1])
        loss.backward()
        opt.step()

    model.eval()
    correct, total = 0, 0
    #problem with the x_val size and type here
    for x_val, y_val in val_dl:


        x_val, y_val = [t for t in (x_val, y_val)]
        x_val = x_val.to(device)
        y_val = y_val.to(device)
        x_val=torch.unsqueeze(x_val,2)
        print(x_val.shape)
        out = model(x_val)
        preds = F.log_softmax(out, dim=1).argmax(dim=1)
        total += y_val.size(0)
        correct += (preds == y_val).sum().item()

    acc = correct / total

    if epoch % 5 == 0:
        print(f'Epoch: {epoch:3d}. Loss: {loss.item():.4f}. Acc.: {acc:2.2%}')

    if acc > best_acc:
        trials = 0
        best_acc = acc
        torch.save(model.state_dict(), 'best.pth')
        print(f'Epoch {epoch} best model saved with accuracy: {best_acc:2.2%}')
    else:
        trials += 1
        if trials >= patience:
            print(f'Early stopping on epoch {epoch}')
            break
    writer.add_scalars('Training vs validation Loss',
                       {'Training' : loss},
                       epoch *len(train_dl)+i)

writer.flush()



Streaming output truncated to the last 5000 lines.
32
1
torch.Size([32, 4000, 1])
torch.int64
shape of the input batch
torch.Size([32, 4000])
torch.Size([1, 32, 100])
32
1
torch.Size([32, 4000, 1])
torch.int64
shape of the input batch
torch.Size([32, 4000])
torch.Size([1, 32, 100])
32
1
torch.Size([32, 4000, 1])
torch.int64
shape of the input batch
torch.Size([14, 4000])
torch.Size([1, 14, 100])
14
1
torch.Size([14, 4000, 1])
torch.int64
torch.Size([32, 4000, 1])
torch.Size([1, 32, 100])
32
1
torch.Size([32, 4000, 1])
torch.Size([32, 4000, 1])
torch.Size([1, 32, 100])
32
1
torch.Size([32, 4000, 1])
torch.Size([32, 4000, 1])
torch.Size([1, 32, 100])
32
1
torch.Size([32, 4000, 1])
torch.Size([32, 4000, 1])
torch.Size([1, 32, 100])
32
1
torch.Size([32, 4000, 1])
torch.Size([32, 4000, 1])
torch.Size([1, 32, 100])
32
1
torch.Size([32, 4000, 1])
torch.Size([32, 4000, 1])
torch.Size([1, 32, 100])
32
1
torch.Size([32, 4000, 1])
torch.Size([12, 4000, 1])
torch.Size([1, 12, 100])
12
1
torch.Size

KeyboardInterrupt: ignored

In [ ]:
!tensorboard --logdir="/content/drive/MyDrive" --bind_all


In [ ]:
model

In [ ]:
model.load_state_dict(torch.load('best.pth'))
model.eval()

'''test_dl = DataLoader(tst_data, batch_size=64, shuffle=False)
test = []
print('Predicting on test dataset')'''

In [ ]:
test_dl = DataLoader(tst_data, batch_size=64, shuffle=False)
test = []
print('Predicting on test dataset')
model = model.to(device)
for batch, _ in tst_data:
    batch=batch.to(device)
    batch = batch.reshape(4000,1,1)
    out = model.train()(batch)
    y_hat = F.log_softmax(out, dim=1).argmax(dim=1)
    test += y_hat.tolist()

In [ ]:
test

In [ ]:
y_hat

In [ ]:
import tensorflow as tf

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()



In [ ]:
a = tf.add(1,2)
b = tf.multiply(a,3)
c = tf.add(4,5)

d = tf.multiply(c,6)
e = tf.multiply(4,5)
f = tf.math.floordiv(c,6)

g = tf.add(b,d)
h = tf.multiply(g,f)
tf.compat.v1.disable_eager_execution()
with tf.compat.v1.Session() as sess:
  writer = tf.summary.FileWriter("./logs", sess.graph)
  print(sess.run(h))
  writer.close()

# New Section